In [ ]:
import pandas as pd

#Read basics.tsv
basics_df = pd.read_csv("datasets/basics.tsv", sep="\t")

#Filter basics_df
basics_df = basics_df[basics_df["titleType"] == "movie"]
basics_df = basics_df[basics_df["genres"] != "\\N"]
# basics_df = basics_df[basics_df["startYear"] != "\\N"]
# basics_df["startYear"] = basics_df["startYear"].apply(int)
# basics_df = basics_df[basics_df["startYear"] >= 1960]
basics_df = basics_df.set_index("tconst")

#Filter akas_df
akas_df = pd.read_csv("datasets/akas.tsv", sep="\t", dtype={
    "titleId": "string", 
    "ordering": int, 
    "title": "string", 
    "region":"string",
    "language":"string"
})

akas_df = akas_df[akas_df["region"].isin(["US","CA"])]
akas_df = akas_df[akas_df["language"] == "en"]
akas_df = akas_df.set_index("titleId")
#Join
akas_basics_df = basics_df.join(akas_df, how='inner')
akas_basics_df


In [33]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [52]:
it_file = open("datasets/it_scripts.txt", "r")
it_data = it_file.read()
it_file.close()
# print(it_data)

coco_file = open("datasets/coco_scripts.txt", "r")
coco_data = coco_file.read()
coco_file.close()
# print(coco_data)

docA = "the car is driven on the road"
docB = "the truck is driven on the highway" 

tfidf = TfidfVectorizer()
response = tfidf.fit_transform([it_data, coco_data])

feature_names = tfidf.get_feature_names()
for col in response.nonzero()[1]:
    print (feature_names[col], ' - ', response[0, col])
    print (feature_names[col], ' - ', response[1, col])

road  -  0.42471718586982765
road  -  0.0
on  -  0.30218977576862155
on  -  0.30218977576862155
driven  -  0.30218977576862155
driven  -  0.30218977576862155
is  -  0.30218977576862155
is  -  0.30218977576862155
car  -  0.42471718586982765
car  -  0.0
the  -  0.6043795515372431
the  -  0.6043795515372431
highway  -  0.0
highway  -  0.42471718586982765
truck  -  0.0
truck  -  0.42471718586982765
on  -  0.30218977576862155
on  -  0.30218977576862155
driven  -  0.30218977576862155
driven  -  0.30218977576862155
is  -  0.30218977576862155
is  -  0.30218977576862155
the  -  0.6043795515372431
the  -  0.6043795515372431
